In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from qamsi.config.trading_config import TradingConfig
from qamsi.runner import Runner
from qamsi.features.preprocessor import Preprocessor
from run import Dataset

(CVXPY) Jun 11 01:32:38 PM: Encountered unexpected exception importing solver PROXQP:
ImportError("dlopen(/Users/buchkovv/qamsi/.venv/lib/python3.12/site-packages/cmeel.prefix/lib/python3.12/site-packages/proxsuite/proxsuite_pywrap.cpython-312-darwin.so, 0x0002): Library not loaded: @rpath/libc++.1.dylib\n  Referenced from: <73C5C23C-530A-3E0E-A88E-7897A0C69618> /Users/buchkovv/qamsi/.venv/lib/python3.12/site-packages/cmeel.prefix/lib/python3.12/site-packages/proxsuite/proxsuite_pywrap.cpython-312-darwin.so\n  Reason: tried: '/Users/runner/miniconda3/envs/proxsuite/lib/libc++.1.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/runner/miniconda3/envs/proxsuite/lib/libc++.1.dylib' (no such file), '/var/folders/0j/bwqcs4y508s2n4ck4dhf3rpc0000gn/T/cmeel-qko5evau/whl/cmeel.prefix/lib/libc++.1.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/var/folders/0j/bwqcs4y508s2n4ck4dhf3rpc0000gn/T/cmeel-qko5evau/whl/cmeel.prefix/lib/libc++.1.dylib' (no such file), '/Users

In [3]:
REBAL_FREQ = "ME"
DATASET = Dataset.SPX_US
ESTIMATION_WINDOW = 365 * 3

In [4]:
experiment_config = DATASET.value()

stocks = tuple(
    pd.read_csv(experiment_config.PATH_OUTPUT / experiment_config.STOCKS_LIST_FILENAME)
    .iloc[:, 0]
    .astype(str)
    .tolist(),
)
experiment_config.ASSET_UNIVERSE = stocks  # type: ignore  # noqa: PGH003

experiment_config.MIN_ROLLING_PERIODS = ESTIMATION_WINDOW + 1
experiment_config.N_LOOKBEHIND_PERIODS = None
experiment_config.REBALANCE_FREQ = REBAL_FREQ

factors = pd.read_csv(experiment_config.PATH_OUTPUT / "factors.csv")
factors["date"] = pd.to_datetime(factors["date"])
factors = factors.set_index("date")
factor_names = tuple(factors.columns.astype(str).tolist())
experiment_config.FACTORS = factor_names

prices = [stock + "_Price" for stock in list(stocks)]
preprocessor = Preprocessor(
    exclude_names=[
        *list(stocks),
        experiment_config.RF_NAME,
        *experiment_config.HEDGING_ASSETS,
        *factor_names,
        *prices,
    ],
)

trading_config = TradingConfig(
    broker_fee=0.05 / 100,
    bid_ask_spread=0.03 / 100,
    total_exposure=1,
    max_exposure=1,
    min_exposure=0,
    trading_lag_days=1,
)

runner = Runner(
    experiment_config=experiment_config,
    trading_config=trading_config,
    verbose=True,
)

Backtest on 2004-03-19 00:00:00 to 2023-12-29 00:00:00


In [57]:
ret = runner.returns.simple_returns.sub(runner.rf, axis=0)
ret.shape

(4980, 1033)

In [58]:
ret = ret.loc["2008-01-01":"2008-12-31"]

n_valid_points = (~ret.isna()).sum(axis=0) / len(ret)
valid_stocks = list(n_valid_points[n_valid_points >= 1.0].index)

ret = ret[valid_stocks]

In [64]:
import numpy as np

fitted_vols = np.eye(ret.shape[1]) * ret.std().to_numpy()
# corr = ret.corr().to_numpy()
cov = ret.cov() / ret.std(axis=0)
cov = cov.to_numpy()

In [65]:
from sklearn.covariance import GraphicalLassoCV
from sklearn.model_selection import TimeSeriesSplit

shrinkage = np.sqrt(np.log(cov.shape[0] - 1) / ret.shape[0])

alphas = np.logspace(-10, -1.5, num=10)
gl = GraphicalLassoCV(
    alphas=alphas,
    cv=TimeSeriesSplit(n_splits=10),
)
gl.fit(cov)
reconstr_cov = gl.covariance_

# reconstr_corr, _ = graphical_lasso(corr, alpha=shrinkage)
# reconstr_corr = reconstr_corr.clip(min=-1, max=1)
# np.fill_diagonal(reconstr_corr, 1)
# a_corr = reconstr_corr.copy()
# np.fill_diagonal(a_corr, 0)
#
# # alphas.min(), gl.alpha_
#
# n = corr.shape[0]
# (a_corr != 0).sum() / 2 / (n * (n - 1) / 2)

alphas.min(), gl.alpha_